In [1]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
import seaborn as sns
sns.set(color_codes=True)
import geopandas as gp
from scipy import stats, integrate

In [2]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOWavailable.csv', encoding='utf-8')
herepath.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW
0,0,998969850F,2014-01-01 13:55,167,186,61.0,0.0,10,Wednesday
1,1,998969850F,2014-01-02 11:05,133,186,60.0,0.0,10,Thursday
2,2,998969850F,2014-01-02 13:20,160,186,61.0,0.0,10,Thursday
3,3,998969850F,2014-01-02 14:30,174,186,52.0,0.0,10,Thursday
4,4,998969850F,2014-01-03 09:05,109,186,57.0,0.0,10,Friday


In [3]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [4]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [5]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [6]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [7]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [8]:
#Visualization on how the function works
us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']

herepath['DATE'] = herepath['DATE_TIME'].str.split(' ').str[0]
r = herepath[herepath['DATE'].isin(us_holidays)]
v = herepath[~herepath.isin(r)]
w = v.dropna()
r.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
0,0,998969850F,2014-01-01 13:55,167,186,61.0,0.0,10,Wednesday,2014-01-01
1099,1099,998969850F,2014-05-26 11:25,137,186,64.5,3.5,30,Monday,2014-05-26
1100,1100,998969850F,2014-05-26 14:15,171,186,76.0,0.0,10,Monday,2014-05-26
1101,1101,998969850F,2014-05-26 15:05,181,186,65.0,0.0,10,Monday,2014-05-26
1102,1102,998969850F,2014-05-26 17:25,209,186,31.0,0.0,10,Monday,2014-05-26


In [9]:
v.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,998969850F,2014-01-02 11:05,133.0,186.0,60.0,0.0,10.0,Thursday,2014-01-02
2,2.0,998969850F,2014-01-02 13:20,160.0,186.0,61.0,0.0,10.0,Thursday,2014-01-02
3,3.0,998969850F,2014-01-02 14:30,174.0,186.0,52.0,0.0,10.0,Thursday,2014-01-02
4,4.0,998969850F,2014-01-03 09:05,109.0,186.0,57.0,0.0,10.0,Friday,2014-01-03


In [10]:
#Proves the data doesn't have any NaN's other than what I specify to drop holidays
np.isnan(herepath.MEAN).unique()

array([False], dtype=object)

In [11]:
numberdays = data['DATE'].unique()
len(numberdays)

NameError: name 'data' is not defined

In [ ]:
data = timeframe2(herepath,0,288,True,Weekday)
epoct = data['LINK_DIR'].value_counts()
epoct
#epoct/(288*len(numberdays))

In [7]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstats(x):
    print('Mean is ', np.mean(x['MEAN']), 'KMH')
    var = ((x['MEAN'].var(ddof=1))+(((x['STDDEV'])**2).sum()/len(x)))
    print('Standard Deviation is ', np.sqrt(var),'KMH')
    print('85th percentile speed is ', x['MEAN'].quantile(0.85,interpolation = 'lower' ), 'KMH')
    print('5th percentile speed is ', x['MEAN'].quantile(0.05,interpolation = 'lower'), 'KMH')
    
    ax = sns.distplot(x['MEAN'], kde=False, bins=70)
    ax.figure.set_size_inches(8,6)
    ax.set_xlim(0,100)
    return ax

In [29]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsGB(x):
    y = pd.DataFrame(columns=['Mean','StDev','FFS','FiveSpd'])
    y['Mean'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())/(0.6213)
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    
    y['StDev'] = (np.sqrt(Var))/(0.6213)
    
    y['FFS'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))/(0.6213)
    
    y['FiveSpd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))/(0.6213)
    
    return y

In [20]:
#For all days and all time periods
bylink = HEREstatsGB(herepath)
bylink.head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
107030432T,62.051514,16.809469,78.0,31.0
107030439T,56.572330,15.981771,72.0,28.0
107030440T,56.113229,16.582816,73.0,27.5
107030444F,58.232521,18.041906,76.0,26.0
107030445F,58.912403,18.107840,77.0,27.0


In [30]:
#for weekdays
Wkd = timeframe('998969850F',herepath,0,288,True,Weekday)
HEREstatsGB(Wkd).head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
998969850F,87.986575,23.886245,111.05746,46.676324


In [22]:
#for weekends
Wend = timeframe('998969850F',herepath,0,288,True,Weekend)
HEREstatsGB(Wend).head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
998969850F,54.666059,14.840524,69.0,29.0


In [27]:
#Weekday Between 6AM and 8PM
Wdtime = timeframe('998969850F',herepath,72,240,True,Weekday)
HEREstatsGB(Wdtime).head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
998969850F,53.58535,14.385467,67.0,29.0


In [24]:
#Stats between 6AM and 9AM
WdtimeAM = timeframe('998969850F',herepath,72,108,True,Weekday)
HEREstatsGB(WdtimeAM).head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
998969850F,54.666059,14.840524,69.0,29.0


In [25]:
#Stats between 9AM and 3PM
WdtimeMD = timeframe('998969850F',herepath,108,180,True,Weekday)
HEREstatsGB(WdtimeMD).head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
998969850F,54.666059,14.840524,69.0,29.0


In [28]:
#Stats between 3PM and 6PM
WdtimePM = timeframe('998969850F',herepath,180,216,True,Weekday)
HEREstatsGB(WdtimePM).head()

,Mean,StDev,FFS,FiveSpd
LINK_DIR,,,,
998969850F,50.929032,15.007642,65.0,27.0
